In [44]:
import equinox as eqx

In [60]:
import os
os.environ["JAX_CACHE_DIR"] = "/cluster/scratch/mpundir/jax-cache"
os.environ["JAX_PLATFORM"] = "cpu"

import jax

jax.config.update("jax_compilation_cache_dir", os.environ["JAX_CACHE_DIR"])
jax.config.update("jax_persistent_cache_min_compile_time_secs", 0)
import jax.numpy as jnp

jax.config.update("jax_enable_x64", True)  # use double-precision
jax.config.update("jax_platforms", "cpu")

import sympy
import symfem

import numpy as np

import matplotlib.pyplot as plt

import itertools
import sys
sys.path.append('/cluster/home/mpundir/dev/femsolver')

from femsolver.plotting import color_scheme

from femsolver.fem_tools import (
    create_finite_element,
    compute_computational_nodes,
    create_basis,
    integrate_over_cell,
    interpolate_at_quadrature,
    batched_jacobian,
)

In [108]:
nodes = np.array([[1, 0], [3, 0], [3, 1], [1, 1]])
connectivity = np.array([0, 1, 2, 3])

In [141]:
degree = 1
element = create_finite_element(
    "quadrilateral", element_type="Lagrange", degree=degree, variant="equispaced"
)

In [156]:
@jax.tree_util.register_pytree_node_class
class Basis:
    """
    A class used to represent a Basis for finite element method (FEM) simulations.

    Attributes
    ----------
    N : jnp.ndarray
        Shape functions evaluated at quadrature points, with shape (nb_quads, nb_nodes_per_element).
    dNdξ : jnp.ndarray
        Derivatives of shape functions with respect to the reference coordinates, with shape (nb_quads, nb_nodes_per_element).
    wts : jnp.ndarray
        Quadrature weights, with shape (nb_quads).

    Methods
    -------
    tree_flatten():
        Flattens the Basis object into a tuple of children and auxiliary data for JAX transformations.
    tree_unflatten(aux_data, children):
        Reconstructs the Basis object from flattened children and auxiliary data.
    """

    def __init__(self, nb_quads, nb_nodes_per_element, dim=1):
        self.N = jnp.zeros((nb_quads, nb_nodes_per_element))
        self.dNdξ = jnp.zeros((dim, nb_quads, nb_nodes_per_element))
        self.d2Ndξ2 = jnp.zeros((dim, dim, nb_quads, nb_nodes_per_element))
        self.wts = jnp.zeros((dim, nb_quads))
        self.quad_pts = jnp.zeros((dim, nb_quads))

    def tree_flatten(self):
        return ((self.N, self.dNdξ, self.wts, self.quad_pts), None)

    @classmethod
    def tree_unflatten(cls, aux_data, children):
        N, dNdξ, ω, quad_pts = children
        instance = cls(N.shape[0], N.shape[1])
        instance.N = N
        instance.dNdξ = dNdξ
        instance.wts = ω
        instance.quad_pts = quad_pts
        return instance


In [157]:
x, y = sympy.symbols("x y")

f = sympy.lambdify([x, y], element.get_basis_functions(), "jax")

# so that output is an array and not list
shape_fun = lambda x, y: jnp.array(f(x, y))

# using JAX JIT on the shape function
shape_functions = jax.jit(shape_fun)

# using forward-mode differentiation to compute the shape derivatives

shape_derivatives_x = jax.jit(jax.jacfwd(shape_functions, argnums=0))
shape_derivatives_y = jax.jit(jax.jacfwd(shape_functions, argnums=1))

# using forward-mode differentiation to compute the 2nd derivatives
shape_double_derivatives_xx = jax.jit(jax.jacfwd(shape_derivatives_x, argnums=0))
shape_double_derivatives_xy = jax.jit(jax.jacfwd(shape_derivatives_x, argnums=1))
shape_double_derivatives_yx = jax.jit(jax.jacfwd(shape_derivatives_y, argnums=0))
shape_double_derivatives_yy = jax.jit(jax.jacfwd(shape_derivatives_x, argnums=1))

# number of computational nodes (based on the polynomial degree)
nb_nodes = len(element.dof_plot_positions())

In [158]:
tdim = element.domain_dim
quadrature_rule = "legendre"
nb_quads = int(
    np.ceil((element.order + 1) / 2)
)  # for exact integration of polynomial degree

basis = Basis(nb_quads**tdim, nb_nodes, dim=tdim)

for i, quad in enumerate(
    list(
        itertools.product(
            symfem.quadrature.get_quadrature(quadrature_rule, nb_quads)[0], repeat=tdim
        )
    )
):

    basis.quad_pts = basis.quad_pts.at[0, i].set(float(quad[0]))
    basis.quad_pts = basis.quad_pts.at[1, i].set(float(quad[1]))
    
    basis.N = basis.N.at[i, :].set(shape_functions(float(quad[0]), float(quad[1])))

    basis.dNdξ = basis.dNdξ.at[0, i, :].set(
        shape_derivatives_x(float(quad[0]), float(quad[1]))
    )
    basis.dNdξ = basis.dNdξ.at[1, i, :].set(
        shape_derivatives_y(float(quad[0]), float(quad[1]))
    )

    basis.d2Ndξ2 = basis.d2Ndξ2.at[0, 0, i, :].set(
        shape_double_derivatives_xx(float(quad[0]), float(quad[1]))
    )
    basis.d2Ndξ2 = basis.d2Ndξ2.at[0, 1, i, :].set(
        shape_double_derivatives_xy(float(quad[0]), float(quad[1]))
    )
    basis.d2Ndξ2 = basis.d2Ndξ2.at[1, 0, i, :].set(
        shape_double_derivatives_yx(float(quad[0]), float(quad[1]))
    )
    basis.d2Ndξ2 = basis.d2Ndξ2.at[1, 1, i, :].set(
        shape_double_derivatives_yy(float(quad[0]), float(quad[1]))
    )


for i, wts in enumerate(
    list(
        itertools.product(
            symfem.quadrature.get_quadrature(quadrature_rule, nb_quads)[1], repeat=tdim
        )
    )
):   
    basis.wts = basis.wts.at[0, i].set(float(wts[0]))
    basis.wts = basis.wts.at[1, i].set(float(wts[1]))

In [159]:
basis.quad_pts.shape

(2, 1)

In [160]:
basis.N.shape

(1, 4)

In [161]:
basis.wts

Array([[1.],
       [1.]], dtype=float64)

In [162]:
nodes[connectivity]

array([[1, 0],
       [3, 0],
       [3, 1],
       [1, 1]])

Forward mapping: We first define 𝑔:ℝ2→ℝ2 with 𝑔|𝑈:𝑈→𝑄 with the bilinear ansatz. It turns out we can easily write this as


$$ (\xi_1, \xi_2) = \phi(x, y) = P_0 + (-P_0 + P_1)x + (-P_0+P_3)y+(P_0 -P_1 + P_2 - P_3)xy$$

[Link](https://math.stackexchange.com/questions/3069262/functions-that-map-a-quadrilateral-to-the-unit-square)

In [163]:
@jax.jit
def map_to_quad_cell(x, cell_nodes):
    p0, p1, p2, p3 = cell_nodes[0], cell_nodes[1], cell_nodes[2], cell_nodes[3]
    mapped = (
        p0
        + (-p0 + p1) * x.at[0].get()
        + (-p0 + p3) * x.at[1].get()
        + (p0 - p1 + p2 - p3) * x.at[0].get() * x.at[1].get()
    )
    return mapped

In [164]:
point = jnp.array([0.0, 1.0])
map_to_quad_cell(point, nodes[connectivity])

Array([1., 1.], dtype=float64)

The jacobian is given as 

$$J = \dfrac{\partial \phi}{\partial \xi}$$

In [165]:
jacobian_fun = jax.jacfwd(map_to_quad_cell)

In [166]:
J = jacobian_fun(point, nodes[connectivity])

In [167]:
print(J)

[[2. 0.]
 [0. 1.]]


In [179]:
# defining dofs
nb_dofs_per_node = 1
total_comp_nodes = nodes.shape[0]
dofs = np.zeros((total_comp_nodes, nb_dofs_per_node))

dofs[:, 0] = nodes[:, 0]**2

In [180]:
@jax.jit
def interpolate_at_quadrature(dofs, N):
    return jnp.einsum("ij, jk...->ik...", N, dofs, optimize="optimal")

@jax.jit
def gradient_at_quadrature(dofs, dN):
    return jnp.einsum("xij, jk...->xik...", dN, dofs, optimize="optimal")


@jax.jit
def integrate_over_cell(value_at_quad, wtsx, wtsy, detJ):
    integral_value = jnp.einsum(
        "i...->...",
        jnp.einsum("i..., i..., i... -> i...", value_at_quad, wtsx, wtsy) * detJ,
        optimize="optimal",
    )

    return integral_value


In [181]:
jnp.sum(
    interpolate_at_quadrature(dofs=dofs, N=basis.N)
    * basis.wts[0][0]
    * basis.wts[1][0]
    * jnp.linalg.det(J)
)

Array(10., dtype=float64)

In [182]:
integrate_over_cell(
    interpolate_at_quadrature(dofs=dofs, N=basis.N),
    basis.wts[0],
    basis.wts[1],
    jnp.linalg.det(J),
)

Array([10.], dtype=float64)

In [193]:
interpolate_at_quadrature(dofs=dofs, N=basis.dNdξ[0, :, :])  

Array([[0.]], dtype=float64)

In [194]:
basis.dNdξ[0, 0, :]@dofs.flatten()

Array(0., dtype=float64)